In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Read in datafile

In [2]:
df = pd.read_csv("rawJailDataIntern.csv")

Create a new variable 'facility_year' that includes both facility and year to help label anomaly detection earlier

In [3]:
df = df.assign(facility_year=df['facility_name']+df['year'].astype(str))

# 2. Explore Missingness

### Missingness Table:

In [4]:
# Total Missing Data:
df.isna().sum()

fips                    0
facility_name           0
year                    0
rated_capacity          7
total_confined_pop      3
adp                    15
confined_women          0
confined_men           12
admissions_year       122
admissions_week        92
admissions_day        159
discharge_year        122
discharge_week         92
discharge_day         160
facility_year           0
dtype: int64

### 2 Takeaways from the missingness table:

1. **Impute yearly counts based on daily and weekly counts**: It seems as if there may be some non-overlapping missingness from the days and weeks counts versus the year counts. Thus, I will try to use the daily and weekly counts to infer the yearly counts for discharge and admissions. 

2. **Impute confined_men using confined_women**: Because this dataset doesn't take into account folks with non-binary gender identities (confined_other doesn't exist in this dataset), it looks like we can fill up some confined_men values by subtracting total_confined_pop by confined_women, given that confined_women doesn't have a count. 

# 3. Impute yearly counts based on daily and weekly counts

After some exploratory data analysis, it's clear that many of the missing yearly values for admissions and discharge often have either day or week data next to them.

As a **rough estimate**, we can impute the missing year columns by multiplying the weekly and daily data that exists for those years. If anomalous, we will remove them later in the code. 

In [5]:
# impute missing admissions_year data based on admissions_week and admissions_day
df['admissions_year'] = np.where(df.admissions_year.isnull(), df.admissions_week * 52, df.admissions_year)
df['admissions_year'] = np.where(df.admissions_year.isnull(), df.admissions_day * 365, df.admissions_year)

# impute missing discharge_year data based on discharge_week and discharge_day
df['discharge_year'] = np.where(df.discharge_year.isnull(), df.discharge_week * 52, df.discharge_year)
df['discharge_year'] = np.where(df.discharge_year.isnull(), df.discharge_day * 365, df.discharge_year)

Now, let's drop the daily and weekly counts because we wish to remove them in the final output. 

In [6]:
df = df.drop(columns=['admissions_day', 'admissions_week', 'discharge_week', 'discharge_day'])

# 4. Impute confined_men using confined_women

Subtract confined_women from total_confined_pop to get confined_men. 

As you can see, doing this reduced the missingness of confined_men to 0.

In [7]:
df['confined_men'] = np.where(df.confined_men.isnull(), df.total_confined_pop - df.confined_women, df.confined_men)
df.confined_men.isna().sum() # how much is still missing from confined_men?

0

# 5. Detect Anomalous Data

Create Percent Change Columns for rated_capacity, total_confined_pop, adp, confined_women, confined_men, admissions_year, discharge_year.

In [8]:
columns = ['rated_capacity', 'total_confined_pop', 'adp', 'confined_women', 'confined_men', 'admissions_year', 'discharge_year']

df = (df.join(df[columns]
              .pct_change(fill_method='ffill'), rsuffix='_pctchange'))

### Pinpoint anomalies:
Here's a table that will help pinpoint anomalies (40%+ difference) that we can use to analyze the original dataset:

In [9]:
anomaly = df.copy()

for column in columns:
    column_pctchange = column + '_pctchange'
    anomaly[column_pctchange] = np.where(abs(anomaly[column_pctchange]) > .40, 
                          "Anomaly", 
                          "Good")
anomaly

,fips,facility_name,year,rated_capacity,total_confined_pop,adp,confined_women,confined_men,admissions_year,discharge_year,facility_year,rated_capacity_pctchange,total_confined_pop_pctchange,adp_pctchange,confined_women_pctchange,confined_men_pctchange,admissions_year_pctchange,discharge_year_pctchange
0,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1985,1864.0,2844.0,2767.0,364,2476.0,93086.0,92834.0,ORANGE COUNTY SHERIFF'S DEPARTMENT1985,Good,Good,Good,Good,Good,Good,Good
1,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1986,2496.0,3217.0,2913.0,375,2842.0,93998.0,93642.0,ORANGE COUNTY SHERIFF'S DEPARTMENT1986,Good,Good,Good,Good,Good,Good,Good
2,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1987,2775.0,3471.0,3250.0,464,3006.0,77747.0,76685.0,ORANGE COUNTY SHERIFF'S DEPARTMENT1987,Good,Good,Good,Good,Good,Good,Good
3,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1989,3478.0,4527.0,4281.0,585,3942.0,125325.0,122631.0,ORANGE COUNTY SHERIFF'S DEPARTMENT1989,Good,Good,Good,Good,Good,Anomaly,Anomaly
4,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1990,3478.0,4402.0,4370.0,510,3892.0,92899.0,93416.0,ORANGE COUNTY SHERIFF'S DEPARTMENT1990,Good,Good,Good,Good,Good,Good,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,56043,WASHAKIE COUNTY DETENTION CENTER,1978,10.0,1.0,NaN,0,1.0,156.0,156.0,WASHAKIE COUNTY DETENTION CENTER1978,Good,Good,Good,Good,Good,Anomaly,Anomaly
163,56043,WASHAKIE COUNTY DETENTION CENTER,1983,10.0,3.0,NaN,1,2.0,413.0,413.0,WASHAKIE COUNTY DETENTION CENTER1983,Good,Anomaly,Good,Anomaly,Anomaly,Anomaly,Anomaly
164,56043,WASHAKIE COUNTY DETENTION CENTER,1988,56.0,12.0,9.0,1,10.0,543.0,537.0,WASHAKIE COUNTY DETENTION CENTER1988,Anomaly,Anomaly,Anomaly,Good,Anomaly,Good,Good
165,56043,WASHAKIE COUNTY DETENTION CENTER,1993,42.0,14.0,16.0,1,13.0,NaN,NaN,WASHAKIE COUNTY DETENTION CENTER1993,Good,Good,Anomaly,Good,Good,Good,Good


### Research Bristol County House of Correction and Jail:

Using the `anomalies`, we can see that Bristol has many anomalous years for its admissions per year variable. 

For instance, Bristol's admissions per year ranges from 1605.0 people/year to 29900.0 people/year. 

Doing some online research, Bristol County House of Correction and Jail has a capacity of 1100 bed facility houses. [<sup>1</sup>](#fn1) Thus, it seems improbable that Bristol County would have a yearly entry of 29900.0 people/year, because each bed would hold 27 people per year if true. Thus, when cleaning the data later using the function, I will pay close attention to making sure that the data that is removed are the numbers closer to 29900.0. 

<span id="fn1"><sup>1</sup>Bristol County Facilities: https://www.bcso-ma.us/facilities.htm</span>

In [10]:
def show_anomaly_table(county, statistic):
    '''
    code to index and show anomalies for a specific county and the county's [insert statistic here]
    
    county - county name (all caps)
    statistic - statistic (i.e. admissions_year, etc.)
    '''
    statistic_pct_change = statistic + '_pctchange'
    return anomaly[anomaly['facility_name'].str.contains(county)][['facility_name',
                                                           'year',
                                                           statistic,
                                                           statistic_pct_change]]

show_anomaly_table('BRISTOL', 'admissions_year')

,facility_name,year,admissions_year,admissions_year_pctchange
105,BRISTOL COUNTY SHERIFFS OFFICE,1985,1605.0,Anomaly
106,BRISTOL COUNTY SHERIFFS OFFICE,1986,1804.0,Good
107,BRISTOL COUNTY SHERIFFS OFFICE,1987,1975.0,Good
108,BRISTOL COUNTY SHERIFFS OFFICE,1989,2085.0,Good
109,BRISTOL COUNTY SHERIFFS OFFICE,1990,2439.0,Good
110,BRISTOL COUNTY SHERIFFS OFFICE,1992,1825.0,Good
111,BRISTOL COUNTY SHERIFFS OFFICE,1994,NaN,Good
112,BRISTOL COUNTY SHERIFFS OFFICE,1995,NaN,Good
113,BRISTOL COUNTY SHERIFFS OFFICE,1996,NaN,Good
114,BRISTOL COUNTY SHERIFFS OFFICE,1997,NaN,Good


# 6. Remove Anomalous Data:

In [11]:
for column in columns:
    column_pctchange = column + '_pctchange'
    df[column] = np.where(abs(df[column_pctchange]) > .40, 
                          np.nan, 
                          df[column])

In [12]:
df = df.iloc[:, 0:10]

df

,fips,facility_name,year,rated_capacity,total_confined_pop,adp,confined_women,confined_men,admissions_year,discharge_year
0,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1985,1864.0,2844.0,2767.0,364.0,2476.0,93086.0,92834.0
1,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1986,2496.0,3217.0,2913.0,375.0,2842.0,93998.0,93642.0
2,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1987,2775.0,3471.0,3250.0,464.0,3006.0,77747.0,76685.0
3,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1989,3478.0,4527.0,4281.0,585.0,3942.0,NaN,NaN
4,6059,ORANGE COUNTY SHERIFF'S DEPARTMENT,1990,3478.0,4402.0,4370.0,510.0,3892.0,92899.0,93416.0
...,...,...,...,...,...,...,...,...,...,...
162,56043,WASHAKIE COUNTY DETENTION CENTER,1978,10.0,1.0,NaN,0.0,1.0,NaN,NaN
163,56043,WASHAKIE COUNTY DETENTION CENTER,1983,10.0,NaN,NaN,NaN,NaN,NaN,NaN
164,56043,WASHAKIE COUNTY DETENTION CENTER,1988,NaN,NaN,NaN,1.0,NaN,543.0,537.0
165,56043,WASHAKIE COUNTY DETENTION CENTER,1993,42.0,14.0,NaN,1.0,13.0,NaN,NaN


# 7. Linear interpolation

Apply linear interpolation to each counties. 

In [13]:
df = df.groupby('fips').apply(lambda group: group.interpolate(method='index'))

Linear Interpolation (after removing anomalies) has reduced missingness to:

In [14]:
df.isna().sum()

fips                   0
facility_name          0
year                   0
rated_capacity         8
total_confined_pop    11
adp                   21
confined_women        10
confined_men          12
admissions_year       18
discharge_year        27
dtype: int64

# 8. Write data to new file 

In [15]:
df.to_csv('cleaned.csv')